In [1]:
%load_ext autotime

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"

time: 590 µs


In [3]:
import pandas as pd
import numpy as np
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.interactions import Interactions
import pandas as pd
from spotlight.evaluation import *
from torch.utils.tensorboard import SummaryWriter
import torch
from torch.utils.tensorboard import SummaryWriter
import logging
from spotlight.factorization.representations import *
import os
import collections

from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.sequence.representations import CNNNet

time: 1.28 s


In [4]:
print(torch.cuda.device_count())

1
time: 648 µs


In [4]:
suffix = "1500K-with-3-vids"
train_data_path = "/home/ec2-user/emb3/data/train-aug-28-phase" + suffix
original_train_data = pd.read_parquet(train_data_path)

#train_data = original_train_data
train_data = original_train_data.copy()
train_data["vindex"] = train_data["vindex"]+1

time: 913 ms


In [7]:
len(original_train_data["vindex"].unique()), train_data["vindex"].max()

(17538, 17539)

time: 376 ms


In [9]:
interactions = Interactions(train_data["uindex"].to_numpy(),
            train_data["vindex"].to_numpy(),
            train_data["pct_cvt"].to_numpy(),
            train_data["latest_watch_time"].to_numpy(),
            num_users=len(original_train_data["uindex"].unique()),
            num_items=(len(original_train_data["vindex"].unique()) + 2) )

time: 870 ms


In [10]:
max_sequence_length=100
min_sequence_length=1
step_size=1
train_seq = interactions.to_sequence(max_sequence_length=max_sequence_length,
                              min_sequence_length=min_sequence_length,
                              step_size=step_size)

time: 1min 10s


In [12]:
uvs = original_train_data.groupby("uindex")["vindex"].agg(list)
train_data = original_train_data[original_train_data.uindex.isin(uvs[uvs.apply(lambda x: len(x)) <= 50].index)]

time: 2min 50s


In [27]:
#len(uvs[uvs.apply(lambda x: len(x)) <= 50]), len(uvs)
len(train_data[train_data.uindex.isin(uvs[uvs.apply(lambda x: len(x)) <= 50].index)]), len(train_data)

(17312359, 24421579)

time: 10.3 s


In [ ]:
hyperparameters = {
    'loss': "adaptive_hinge",
    'batch_size': 256,
    'learning_rate': 1e-3,
    'l2': 1e-06,
    'n_iter': 30,
    'embedding_dim': 64
}

model_alias = ",".join([k+"="+str(v) for k,v in collections.OrderedDict(hyperparameters).items()])
model_alias = model_alias + "__TMP"
tensorboard_base_dir="/home/ec2-user/emb3/ttest"
writer = SummaryWriter(log_dir='{}/{}'.format(tensorboard_base_dir, model_alias))

model_store_dir= "/home/ec2-user/emb3/models"

def notify_loss_completion(epoch_id, batch_id, loss, net, model):
    #print("notify_loss_completion")
    writer.add_scalar("Batch/loss", loss, batch_id)
    print('[Epoch {}] Batch {}, Loss {}'.format(epoch_id, batch_id, loss))
    logging.info('[Epoch {}] Batch {}, Loss {}'.format(epoch_id, batch_id, loss))

def notify_batch_eval_completion(epoch_id, batch_id, loss, net, model):
    #print("notify_batch_eval_completion")
#     pairs_ndcg = nn_pairs_ndcg_score(net)
#     writer.add_scalar("Batch/pairs_ndcg", pairs_ndcg, batch_id)
#     logging.info('[Epoch {}] Batch {}, Embs NDCG = {:.4f}'.format(epoch_id, batch_id, pairs_ndcg))
#     print('[Epoch {}] Batch {}, Embs NDCG = {:.4f}'.format(epoch_id, batch_id, pairs_ndcg))
    print("Here")
    net.train()
    
def notify_epoch_completion(epoch_num, total_loss, net, model):
    #print("notify_epoch_completion")
    writer.add_scalar("Epoch/loss", total_loss, epoch_num)
    pairs_ndcg = nn_pairs_ndcg_score(net)
    writer.add_scalar("Epoch/pairs_ndcg", pairs_ndcg, epoch_num)
#     hit_ratio, ndcg = evaluate_hit_ratio_and_ndcg(model)
#     writer.add_scalar("Epoch/HR", hit_ratio, epoch_num)
#     writer.add_scalar("Epoch/NDCG", ndcg, epoch_num)
    hit_ratio, ndcg = -1,-1
    logging.info('******** [Epoch {}]  Embs NDCG {:.4f}, Hit Ratio: {:.4f}, NDCG: {:.4f}'.format(epoch_num,
                                                                                                    pairs_ndcg,
                                                                                                    hit_ratio,
                                                                                                    ndcg))
    print('******** [Epoch {}]  Embs NDCG {:.4f}, Hit Ratio: {:.4f}, NDCG: {:.4f}'.format(epoch_num,
                                                                                                    pairs_ndcg,
                                                                                                    hit_ratio,
                                                                                                    ndcg))
    torch.save(net, model_store_dir + "/" + model_alias + "-" + str(epoch_num))
    net.train()


os.environ["SUFFIX"] = suffix
os.environ["BASE_DIR"] = "/home/ec2-user/emb3"

writer.add_text('alias', model_alias, 0)

CUDA=True
h = hyperparameters
random_state = np.random.RandomState(100)

model = ImplicitSequenceModel(loss=h['loss'],
                              representation='lstm',
                              batch_size=h['batch_size'],
                              learning_rate=h['learning_rate'],
                              l2=h['l2'],
                              n_iter=h['n_iter'],
                              embedding_dim=h['embedding_dim'],
                              use_cuda=CUDA,
                              random_state=random_state,
                             notify_loss_completion=notify_loss_completion,
                                  notify_batch_eval_completion=notify_batch_eval_completion,
                                  notify_epoch_completion=notify_epoch_completion,
                             log_loss_interval=5000,
                             log_eval_interval=20000)

model.fit(train_seq, verbose=True)

[Epoch 0] Batch 0, Loss 1.008164405822754
Here
[Epoch 0] Batch 5000, Loss 0.4232798805832863
[Epoch 0] Batch 10000, Loss 0.2841775295913219
[Epoch 0] Batch 15000, Loss 0.2548156420081854
[Epoch 0] Batch 20000, Loss 0.2421273371309042
Here
[Epoch 0] Batch 25000, Loss 0.2349102160125971
[Epoch 0] Batch 30000, Loss 0.2297738703995943
Epoch 0: loss 8711.137982949615


/home/ec2-user/emb3/spotlight/spotlight/evaluation.py:71: RuntimeWarning: invalid value encountered in true_divide
  embeds_norm = np.divide(embeds, np.sqrt(np.square(embeds).sum(axis=1)).reshape(-1, 1))


In [7]:
m = torch.load("models/64-4-bpr-MLP-suf-500K-with-3-vids-loss-adaptive_bpr-lr-0.001-l2-1e-05-mom-0.99-smpl-1.0-ng-3-bt-1024-drp-None-amsgrad-True-adamw=None-0")

time: 1.88 s


In [15]:
m.embedding_item.weight[1:,:].shape, m.embedding_item.weight.shape

(torch.Size([17388, 128]), torch.Size([17389, 128]))

time: 2.62 ms


In [6]:
suffix = "500K-with-3-vids"
os.environ["SUFFIX"] = suffix
os.environ["BASE_DIR"] = "/home/ec2-user/emb3"

time: 699 µs


In [8]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

dataset = get_movielens_dataset(variant='1M')

train, test = random_train_test_split(dataset)


time: 150 ms


In [9]:
model = ImplicitFactorizationModel(n_iter=15,
                                   loss='bpr')
model.fit(train)

mrr = mrr_score(model, test)

time: 6min 8s


In [10]:
mrr.mean()

0.02882165588353406

time: 2.52 ms


In [ ]:
NUM_SAMPLES = 100

LEARNING_RATES = [1e-3, 1e-2, 5 * 1e-2, 1e-1]
LOSSES = ['bpr', 'hinge', 'adaptive_hinge', 'pointwise']
BATCH_SIZE = [8, 16, 32, 256]
EMBEDDING_DIM = [8, 16, 32, 64, 128, 256]
N_ITER = list(range(5, 20))
L2 = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.0]


def sample_cnn_hyperparameters(random_state, num):

    space = {
        'n_iter': N_ITER,
        'batch_size': BATCH_SIZE,
        'l2': L2,
        'learning_rate': LEARNING_RATES,
        'loss': LOSSES,
        'embedding_dim': EMBEDDING_DIM,
        'kernel_width': [3, 5, 7],
        'num_layers': list(range(1, 10)),
        'dilation_multiplier': [1, 2],
        'nonlinearity': ['tanh', 'relu'],
        'residual': [True, False]
    }

    sampler = ParameterSampler(space,
                               n_iter=num,
                               random_state=random_state)

    for params in sampler:
        params['dilation'] = list(params['dilation_multiplier'] ** (i % 8)
                                  for i in range(params['num_layers']))

        yield params


def sample_lstm_hyperparameters(random_state, num):

    space = {
        'n_iter': N_ITER,
        'batch_size': BATCH_SIZE,
        'l2': L2,
        'learning_rate': LEARNING_RATES,
        'loss': LOSSES,
        'embedding_dim': EMBEDDING_DIM,
    }

    sampler = ParameterSampler(space,
                               n_iter=num,
                               random_state=random_state)

    for params in sampler:

        yield params

In [14]:
max_sequence_length=200
min_sequence_length=2
step_size=2

train_seq = train.to_sequence(max_sequence_length=max_sequence_length,
                              min_sequence_length=min_sequence_length,
                              step_size=step_size)
test_seq = test.to_sequence(max_sequence_length=max_sequence_length,
                        min_sequence_length=min_sequence_length,
                        step_size=step_size)

time: 1.6 s
